In [140]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

In [141]:
BASE_DIR = r"C:\Users\Administrator.DESKTOP-10M2D22\Desktop"
GLOVE_DIR = os.path.join(BASE_DIR,"glove.6B")
TEXT_DATA_DIR = os.path.join(BASE_DIR,"20_newsgroup")
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [142]:
#first,build index mapping words in embedding set to their embedding vector
embeddings_index = {}
with open(os.path.join(GLOVE_DIR,"glove.6B.100d.txt"),"rb") as f:
    for line in f:
        word,coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep=" ")
        embeddings_index[word] = coefs
print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [123]:
#second,prepare text sampels and their labels
texts = []
labels_index = {}
labels = []
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR,name)
    if os.path.isdir(path):
        label_id  = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path,fname)
                args ={}  if sys.version_info <(3,) else{'encoding': 'latin-1'}
                with open(fpath,**args) as f:
                    t = f.read()
                    i = t.find("\n\n")
                    if 0<i:
                        t =t[1:]
                    texts.append(t)
                labels.append(label_id)
print("Found %s texts." % len(texts))

Found 19997 texts.


In [124]:
#finally,vectorize the text sampels into a 2D integer tensor
tokenizer = Tokenizer(nb_words = MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

data = pad_sequences(sequences,maxlen = MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print("Shape of dtat tensor:",data.shape)
print("Shape of label tensor:",labels.shape)

Found 214876 unique tokens.
Shape of dtat tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [131]:
#split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
num_validation_samples = int(VALIDATION_SPLIT*data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [132]:
#preapre embedding matrix
num_words = min(MAX_NUM_WORDS,len(word_index)+1)
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))
for word,i in word_index.items():
    if i>=MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [133]:
#load pre_train word embeddings into a Embedding layer
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Training model...")

Training model...


In [134]:
#train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype="int32")
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128,5,activation="relu")(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128,5,activation="relu")(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128,5,activation="relu")(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128,activation="relu")(x)
preds = Dense(len(labels_index),activation="softmax")(x)

model = Model(sequence_input,preds)
model.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["acc"])

model.fit(X_train,y_train,
         batch_size=128,
         epochs=10,
         validation_data=(X_val,y_val))

W0729 15:22:47.859762 30824 deprecation.py:323] From D:\software\anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0729 15:22:48.013304 30824 deprecation_wrapper.py:119] From D:\software\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 69s 4ms/step - loss: 2.9766 - acc: 0.0604 - val_loss: 3.1523 - val_acc: 0.0000e+00
Epoch 2/10
15998/15998 [==============================] - 68s 4ms/step - loss: 2.9461 - acc: 0.0613 - val_loss: 3.2985 - val_acc: 0.0000e+00
Epoch 3/10
15998/15998 [==============================] - 69s 4ms/step - loss: 2.9209 - acc: 0.0581 - val_loss: 3.4424 - val_acc: 0.0000e+00
Epoch 4/10
15998/15998 [==============================] - 69s 4ms/step - loss: 2.8999 - acc: 0.0577 - val_loss: 3.5845 - val_acc: 0.0000e+00
Epoch 5/10
15998/15998 [==============================] - 68s 4ms/step - loss: 2.8821 - acc: 0.0586 - val_loss: 3.7254 - val_acc: 0.0000e+00
Epoch 6/10
15998/15998 [==============================] - 66s 4ms/step - loss: 2.8671 - acc: 0.0589 - val_loss: 3.8654 - val_acc: 0.0000e+00
Epoch 7/10
15998/15998 [==============================] - 64s 4ms/step - loss: 2.8544 - acc: 0.0584 - val